I have a csv file with all the date/times of ASTER observations I'm using, and I want to now download all the MODIS/Terra Snow Cover (Daily L3 Global 500m SIN Grid, Version 61; MOD10A1) imagery for the same date/times covering the same spot.

I'm using the Earthdata API because doing this through their web portal would be a slow and cumbersome process.

Some references:
 * [Earthdata API](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html)
 * [MOD10A1 product](https://nsidc.org/data/MOD10A1/versions/61)

In [1]:
import requests
import pandas as pd
import io
import glob
#import aster_utils

Look at the ASTER images that I have saved, make a list of them and their timestamps:

In [2]:
## Specify the directory all my ASTER images are in
#aster_directory = "/storage/spestana/ASTER/AST_L1T/geotiff/T/T_band14_Tuolumne-and-CUES/"
#
## Find all our ASTER files and their timestamps in our directory
#aster = aster_utils.aster_timestamps(aster_directory, ext='tif')
#
## Add a UTC-8 datetime for pacific standard that CUES uses
#aster['datetime'] = aster.timestampUTC - pd.Timedelta(hours=8)
#
## Save a csv file of the ASTER files we are using
##aster.to_csv('aster_AST_L1T_band14_Tuolumne-and-CUES.csv')
#
# Alternatively, if I already have a csv file somewhere with the ASTER timestamps I want, read it in
aster = pd.read_csv('../aster_AST_L1T_band14_Tuolumne-and-CUES.csv')

Set up some of the granule search criteria for Earthdata search API

In [3]:
url = "https://cmr.earthdata.nasa.gov/search/granules.csv?" #"https://cmr.earthdata.nasa.gov/search/collections?has_granules_created_at\[\]=2015-01-01T19:00:00Z,"

# Product information for the MODIS products I want
shortname_id = "short_name=MOD10A1" # "short_name=MOD03"
version = "version=61"

# bounding box around the study area I want to look at (upper Tuolumne River basin)
# lower left longitude, lower left latitude, upper right longitude, upper right latitude
bounding_box = "bounding_box[]=-120,37,-118,38" 

In [4]:
# empty list to hold modis download URLs
modis_urls = []

for i, this_timestamp in enumerate(aster.timestampUTC):
       
    # the date is the first 10 characters of the timestamp string
    date = str(this_timestamp)[:10]
    # hour follows a space after the date
    hour_start = str(this_timestamp).split(" ")[1][:2]
    # and search until 1 hour later
    hour_end = int(hour_start) + 1
    
    start_time = "{date}T{hour_start}:00:00Z".format(date=date,hour_start=hour_start)
    end_time = "{date}T{hour_end}:00:00Z".format(date=date,hour_end=hour_end)
    time_range = "temporal=" + start_time + "," + end_time
    
    # build the whole request URL and make the request.get
    request_url = url+"&"+shortname_id+"&"+version+"&"+bounding_box+"&"+time_range
    response = requests.get(request_url)
    # read the response CSV and put in a temporary dataframe
    df = pd.read_csv(io.StringIO(response.text))
    
    for modis_url in df['Online Access URLs']:
        modis_urls.append(modis_url)

In [5]:
modis_urls

['https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2017.03.04/MOD10A1.A2017063.h08v05.061.2021267023554.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2017.03.20/MOD10A1.A2017079.h08v05.061.2021268101357.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2017.04.21/MOD10A1.A2017111.h08v05.061.2021270213842.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2019.01.13/MOD10A1.A2019013.h08v05.061.2020284120312.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2019.03.26/MOD10A1.A2019085.h08v05.061.2020291025157.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2019.04.27/MOD10A1.A2019117.h08v05.061.2020292205834.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2019.05.13/MOD10A1.A2019133.h08v05.061.2020293235606.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2019.05.28/MOD10A1.A2019148.h08v05.061.2020295151500.hdf',
 'https://n5eil01u.ecs.nsidc.org/DP4/MOST/MOD10A1.061/2019.05.28/MOD10A1.A2019148.h08v05.061.202

Write the list of URLs out to a text file:

In [6]:
with open("MOD10A1_download_list.txt", "w") as output:
    for row in modis_urls:
        output.write(str(row) + '\n')

**Download using wget and this text file list of all the MODIS observations**

`wget --http-user=YOUR_USERNAME --ask-password --keep-session-cookies --auth-no-challenge=on -c -i modis_download_list.txt`

---